<a href="https://colab.research.google.com/github/debanjan-mondal-2005/University-Student-Query-Priority-Dataset/blob/main/Classification_University_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/coderanand/university-query-priority-classification")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: debanjanmondal2314
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/coderanand/university-query-priority-classification


100%|██████████| 47.8k/47.8k [00:00<00:00, 58.2MB/s]

In [13]:
train_data = pd.read_csv("/content/university-query-priority-classification/university_query_train.csv")
test_data = pd.read_csv("/content/university-query-priority-classification/university_query_test.csv")

In [14]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Query_ID          5000 non-null   int64 
 1   Student_Query     5000 non-null   object
 2   Department        5000 non-null   object
 3   Days_To_Deadline  5000 non-null   int64 
 4   Priority_Label    5000 non-null   object
dtypes: int64(2), object(3)
memory usage: 195.4+ KB


In [15]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Query_ID          1000 non-null   int64 
 1   Student_Query     1000 non-null   object
 2   Department        1000 non-null   object
 3   Days_To_Deadline  1000 non-null   int64 
 4   Priority_Label    1000 non-null   object
dtypes: int64(2), object(3)
memory usage: 39.2+ KB


In [16]:
train_data.sample()

,Query_ID,Student_Query,Department,Days_To_Deadline,Priority_Label
705,381,Please share syllabus for Data Science course.,Finance Office,24,Low


In [19]:
train_data

,Student_Query,Department,Days_To_Deadline,Priority_Label
0,How to join student clubs?,Examination Cell,31,Low
1,My admit card has incorrect details.,Examination Cell,48,High
2,How to reset my university portal password?,Finance Office,5,Medium
3,My exam form is not submitted and tomorrow is ...,Academic Office,18,High
4,I cannot download my hall ticket for tomorrow'...,Administration,43,High
...,...,...,...,...
4995,What are university working hours?,Academic Office,21,Low
4996,Fee payment deadline is today but portal is no...,Hostel Office,44,High
4997,What are university working hours?,Library,24,Low
4998,How to reset my university portal password?,IT Support,27,Medium


In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Concatenate
from tensorflow.keras.utils import to_categorical

In [23]:
label_encoder = LabelEncoder()
train_data['Priority_Label'] = label_encoder.fit_transform(train_data['Priority_Label'])

y_train = to_categorical(train_data['Priority_Label'])

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Define parameters
num_words = 5000
max_length = 12
oov_token = "<OOV>"

# Initialize tokenizer
tokenizer = Tokenizer(
    num_words=num_words,
    oov_token=oov_token
)

# Fit tokenizer on training data only
tokenizer.fit_on_texts(train_data['Student_Query'])

# Convert text to sequences
X_train_text = tokenizer.texts_to_sequences(train_data['Student_Query'])
X_test_text  = tokenizer.texts_to_sequences(test_data['Student_Query'])

# Apply padding
X_train_text = pad_sequences(
    X_train_text,
    maxlen=max_length,
    padding='post',
    truncating='post'
)

X_test_text = pad_sequences(
    X_test_text,
    maxlen=max_length,
    padding='post',
    truncating='post'
)

In [28]:
dept_encoder = OneHotEncoder(sparse_output=False)
X_train_dept = dept_encoder.fit_transform(train_data[['Department']])
X_test_dept = dept_encoder.transform(test_data[['Department']])

In [30]:
X_train_days = train_data[['Days_To_Deadline']].values
X_test_days = test_data[['Days_To_Deadline']].values

In [31]:
# Text input
text_input = Input(shape=(max_length,))
embedding = Embedding(input_dim=10000, output_dim=128)(text_input)
lstm_out = LSTM(64)(embedding)

# Department input
dept_input = Input(shape=(X_train_dept.shape[1],))

# Days input
days_input = Input(shape=(1,))

# Concatenate all
combined = Concatenate()([lstm_out, dept_input, days_input])

dense1 = Dense(64, activation='relu')(combined)
output = Dense(3, activation='softmax')(dense1)

model = Model(inputs=[text_input, dept_input, days_input], outputs=output)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 12)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 12, 128)   │  1,280,000 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 64)        │     49,408 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 72)        │          0 │ lstm[0][0],       │
│ (Concatenate)       │                   │            │ input_layer_1[0]… │
│                     │                   │            │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      4,672 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3)         │        195 │ dense[0][0]       │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,334,275 (5.09 MB)

 Trainable params: 1,334,275 (5.09 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model.fit(
    [X_train_text, X_train_dept, X_train_days],
    y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - accuracy: 0.6552 - loss: 1.0665 - val_accuracy: 1.0000 - val_loss: 8.0101e-04
Epoch 2/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 1.0000 - loss: 4.6676e-04 - val_accuracy: 1.0000 - val_loss: 7.1094e-05
Epoch 3/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 1.0000 - loss: 5.1488e-05 - val_accuracy: 1.0000 - val_loss: 2.7901e-05
Epoch 4/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 1.0000 - loss: 2.2798e-05 - val_accuracy: 1.0000 - val_loss: 1.4521e-05
Epoch 5/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 1.0000 - loss: 1.1783e-05 - val_accuracy: 1.0000 - val_loss: 8.7739e-06
Epoch 6/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 1.0000 - loss: 7.4665e-06 - val_accuracy: 1.0000 - val_loss: 5.7210e-06
Epoch 7/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 29ms/step - accuracy: 1.0000 - loss: 5.2977e-06 - val_accuracy: 1.0000 - val_loss: 3.9200e-06
Epoch 8/10
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 20

In [33]:
predictions = model.predict(
    [X_test_text, X_test_dept, X_test_days]
)

predicted_labels = np.argmax(predictions, axis=1)
predicted_labels = label_encoder.inverse_transform(predicted_labels)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


In [35]:
import numpy as np

# 1️⃣ Text preprocessing (use same tokenizer & max_length used during training)
X_test_text = tokenizer.texts_to_sequences(test_data['Student_Query'])

X_test_text = pad_sequences(
    X_test_text,
    maxlen=max_length,
    padding='post',
    truncating='post'
)

# 2️⃣ Encode Department (use same encoder fitted on train data)
X_test_dept = dept_encoder.transform(test_data[['Department']])

# 3️⃣ Days_To_Deadline (numerical feature)
X_test_days = test_data[['Days_To_Deadline']].values

# 4️⃣ Predict probabilities
predictions = model.predict(
    [X_test_text, X_test_dept, X_test_days]
)

# 5️⃣ Convert probabilities to class index
predicted_classes = np.argmax(predictions, axis=1)

# 6️⃣ Convert numeric labels back to original category names
predicted_labels = label_encoder.inverse_transform(predicted_classes)

# 7️⃣ Add predictions to dataframe
test_data['Predicted_Priority'] = predicted_labels

# 8️⃣ Save results (optional)
test_data.to_csv("test_predictions.csv", index=False)

print("Prediction Completed.")

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
Prediction Completed.


In [39]:
import numpy as np

# Select only first row (keep dimensions correct)
first_text = X_test_text[0:1]
first_dept = X_test_dept[0:1]
first_days = X_test_days[0:1]

# Predict
prediction = model.predict([first_text, first_dept, first_days])

# Convert probability → class
predicted_class = np.argmax(prediction, axis=1)

# Convert back to original label
predicted_label = label_encoder.inverse_transform(predicted_class)

print("Predicted Priority:", predicted_label[0])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
Predicted Priority: Medium


In [ ]:
import numpy as np

# Select only first row (keep dimensions correct)
first_text = X_test_text[0:1]
first_dept = X_test_dept[0:1]
first_days = X_test_days[0:1]

# Predict
prediction = model.predict([first_text, first_dept, first_days])

# Convert probability → class
predicted_class = np.argmax(prediction, axis=1)

# Convert back to original label
predicted_label = label_encoder.inverse_transform(predicted_class)

print("Predicted Priority:", predicted_label[0])